In [1]:
from smtutil import hiveutil, oracle_to_hive
from datetime import datetime, timedelta
import re, glob, shutil, os
from shutil import copyfile
from pathlib import Path
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import lag, lead, first, last, udf, lit, when, expr
import pyspark.sql.functions as F
import yaml
import argparse

byte_to_kb_ratio=1024

# schema
schema_raw = StructType([
    StructField("MSISDN", StringType(), True),
    StructField("IP1", StringType(), True),
    StructField("TIME_START", StringType(), True),
    StructField("IP2", StringType(), True),
    StructField("BYTE_COUNT_UP", IntegerType(), True),
    StructField("BYTE_COUNT_DOWN", IntegerType(), True),
    StructField("SSL_CERT_DOMAIN", StringType(), False),
    StructField("TIME_END", StringType(), True)])

def move_file(input_dir, output_dir, filename_format):

    for f in Path(input_dir).glob(filename_format): # grabs all files
        f.rename(Path(output_dir).joinpath(f.name)) # moves to target folder.

    print(f'[Info] Moved files from {input_dir} to {output_dir} with file format {filename_format}')
    return

In [2]:
time_stamp = '20221207'

In [3]:
# # get binary time string
# def hourly_string(l):
#     output = ''
#     for i in range(0, 24):
#         if int(i) in l:
#             output += '1'
#         else:
#             output += '0'
#     return output
# # define pyspark udf
# udf_hourly_string = F.udf(lambda x: hourly_string(x), StringType())

# get binary time string
def xmins_string(l, interval=60):
    output = ''
    for i in range(0, 24*60//interval):
        if int(i) in l:
            output += '1'
        else:
            output += '0'
    return output

# define pyspark udf
udf_hourly_string = F.udf(lambda x: xmins_string(x, 60), StringType())
udf_15mins_string = F.udf(lambda x: xmins_string(x, 15), StringType())
udf_30mins_string = F.udf(lambda x: xmins_string(x, 30), StringType())

# binary string to integer
udf_b2int = F.udf(hiveutil.binary2int)

In [4]:
# get spark session 
sc, spark = hiveutil.get_sc_spark()

In [5]:
working_dir = '/app/ws-simon/data/weblog_testing_20221207/'
# time_stamp,
# filename_format,
# input_dir,
# pipeline_output_dir,
# derived_dir,
# byte_to_kb_ratio=1024

In [6]:
### 1. -------------------------------------- Move file to working directory --------------------------------------
# move_file(input_dir, working_dir, filename_format)

### 2. -------------------------------------- Read data file --------------------------------------
print(f'[Info] Reading Data Files.....')
print(datetime.now())

# print(f'[Info] Read csv files of timestamp {time_stamp}')
# read csv
sdf = spark.read.csv(f"{working_dir}", sep='|', header=False, schema=schema_raw)
print(f'[Info] df size: {sdf.count()}, {len(sdf.columns)}')

[Info] Reading Data Files.....
2022-12-12 10:27:47.235159
[Info] df size: 2907771261, 8


In [7]:
sdf2 = sdf.alias('1')

In [8]:
# sdf = sdf2.alias('1')

In [9]:
# drop na domain rows
sdf = sdf.na.drop(subset=['SSL_CERT_DOMAIN'])
# print(f'[Info] df size after dropped na in Domain Column: {sdf.count()}, {len(sdf.columns)}')

# print(f'[Info] End of Reading Data Files')
# print(datetime.now())

### 3. -------------------------------------- Transform Data and Create Columns --------------------------------------

# create necessary columns for grouping
sdf = sdf.withColumn('ACCESS_DATE', F.to_date(F.col('TIME_START'), 'yyyyMMddHHmmss')) \
                     .withColumn('SSL_CERT_DOMAIN', F.upper(F.col('SSL_CERT_DOMAIN'))) \
                     .withColumn('START_TIMESTAMP', F.to_timestamp(F.col('TIME_START'),'yyyyMMddHHmmss')) \
                     .withColumn('END_TIMESTAMP', F.to_timestamp(F.col('TIME_END'),'yyyyMMddHHmmss')) \
                     .withColumn('SECONDS', F.col('END_TIMESTAMP').cast('long') - F.col('START_TIMESTAMP').cast('long')) \
                     .withColumn('MINUTES', F.col('SECONDS')/60) \
                     .withColumn('START_HOUR', F.hour(F.col('START_TIMESTAMP'))) \
                     .withColumn('START_INTERVAL_15MIN', F.floor(((F.substring(F.col('TIME_START'), 9,2).cast('int') * 60 + F.substring(F.col('TIME_START'), 11,2).cast('int'))/15))) \
                     .withColumn('START_INTERVAL_30MIN', F.floor(((F.substring(F.col('TIME_START'), 9,2).cast('int') * 60 + F.substring(F.col('TIME_START'), 11,2).cast('int'))/30)))

# add interval col
# interval_to_cut = [15, 30]
# for i in interval_to_cut:
#     sdf = sdf.withColumn(f'START_INTERVAL_{i}min', (F.substring(F.col('TIME_START'), 9,10).cast(int) * 60 + F.substring(F.col('TIME_START'), 11,12).cast(int))//i)

# group by SUBR_NUM, ACCESS_DATE, SSL_CERT_DOMAIN
sdf = sdf.groupBy('MSISDN', 'ACCESS_DATE', 'SSL_CERT_DOMAIN').agg(
                F.count("*").alias('TOTAL_HIT'), \
                F.collect_set(F.col('START_HOUR')).alias('START_HOUR_SET'), \
                F.sum('BYTE_COUNT_UP').alias('BYTE_COUNT_UP'), \
                F.sum('BYTE_COUNT_DOWN').alias('BYTE_COUNT_DOWN'), \
                F.sum('SECONDS').alias('SECONDS'), \
                F.sum('MINUTES').alias('MINUTES'), \
                F.collect_set(F.col('START_INTERVAL_15MIN')).alias('START_INTERVAL_15MIN_SET'), \
                F.collect_set(F.col('START_INTERVAL_30MIN')).alias('START_INTERVAL_30MIN_SET'), \
)

In [10]:
sdf.limit(1).toPandas()

MSISDN ACCESS_DATE        SSL_CERT_DOMAIN  TOTAL_HIT  \
0  85219100002814  2022-12-07  CFCDN.APOWERSOFT.INFO          6   

  START_HOUR_SET  BYTE_COUNT_UP  BYTE_COUNT_DOWN  SECONDS  MINUTES  \
0           [14]          81359           488231      861    14.35   

  START_INTERVAL_15MIN_SET START_INTERVAL_30MIN_SET  
0                     [57]                     [28]

In [11]:
# sdf.count()

In [12]:
# add last few columns
sdf = sdf.withColumn('SUBR_NUM', when(F.col('MSISDN').startswith('852'), expr("substring(MSISDN, 4, length(MSISDN))")).otherwise(F.col('MSISDN'))) \
                     .withColumn('BYTE_COUNT', sum([F.col('BYTE_COUNT_UP'), F.col('BYTE_COUNT_DOWN')])) \
                     .withColumn('KB_COUNT', F.col('BYTE_COUNT')/byte_to_kb_ratio) \
                     .withColumn('KB_COUNT_UP', F.col('BYTE_COUNT_UP')/byte_to_kb_ratio) \
                     .withColumn('KB_COUNT_DOWN', F.col('BYTE_COUNT_DOWN')/byte_to_kb_ratio) \
                     .withColumn('HOUR_BINARY', udf_hourly_string(F.col('START_HOUR_SET'))) \
                     .withColumn('date_id', F.date_format(F.col('ACCESS_DATE'), "yyyyMMdd")) \
                     .withColumn('15MIN_BINARY', udf_15mins_string(F.col('START_INTERVAL_15MIN_SET'))) \
                     .withColumn('30MIN_BINARY', udf_30mins_string(F.col('START_INTERVAL_30MIN_SET')))

# reorder
#     sdf = sdf.select('SUBR_NUM', 'MSISDN', 'ACCESS_DATE', 'SSL_CERT_DOMAIN', 'BYTE_COUNT', 'KB_COUNT', 'HOUR_BINARY', 'TOTAL_HIT', 'BYTE_COUNT_UP', 'KB_COUNT_UP', 'BYTE_COUNT_DOWN', 'KB_COUNT_DOWN', 'SECONDS', 'MINUTES', 'date_id')
sdf = sdf.select(F.col('SUBR_NUM').cast("bigint").alias("subr_num"), \
                 F.col('SSL_CERT_DOMAIN').alias("ssl_cert_domain"), \
                 F.col('BYTE_COUNT').cast("bigint").alias("byte_count"), \
                 udf_b2int(F.col('HOUR_BINARY')).cast("int").alias("hour_binary"), \
                 F.col("TOTAL_HIT").cast("bigint").alias("total_hit"), \
                 'date_id', \
                 udf_b2int(F.col('15MIN_BINARY')).cast("int").alias("15min_binary"), \
                 udf_b2int(F.col('30MIN_BINARY')).cast("int").alias("30min_binary"),
                )

In [13]:
# filter only today's data
sdf3 = sdf.filter(F.col('date_id') == time_stamp)

# lower case column names
#     for col in sdf3.columns:
#         sdf3 = sdf3.withColumnRenamed(col, col.lower())

In [14]:
sdf.count()

366892197

In [15]:
sdf3.count()

278449056

In [16]:
sdf3.limit(5).toPandas()

subr_num           ssl_cert_domain  byte_count  hour_binary  total_hit  \
0  19100002814     CFCDN.APOWERSOFT.INFO      569590          512          6   
1  19100014064      CDN2.SMOOT.APPLE.COM       17243          512          1   
2  19100014064              FACEBOOK.COM        6009            4          1   
3  19100014064  PUBADS.G.DOUBLECLICK.NET        6015           32          1   
4  19100025837   BAICHUAN-SDK.ALICDN.COM       60183        11776          6   

    date_id  15min_binary  30min_binary  
0  20221207           NaN        524288  
1  20221207           NaN        262144  
2  20221207        1024.0            32  
3  20221207     1048576.0          1024  
4  20221207           NaN      77070336

In [ ]:
# # filter only today's data
# sdf = sdf.filter(F.col('date_id') == time_stamp)

# # lower case column names
# #     for col in sdf.columns:
# #         sdf = sdf.withColumnRenamed(col, col.lower())

In [17]:
# ### 4. -------------------------------------- Save to parquet --------------------------------------

# # print(f'[Info] Start Transforming Data......')
# # print(datetime.now())

# # save to parquet
sdf3.write.mode('append').partitionBy('date_id').parquet('/app/sna_bdmrdev03/ws-simon/data/weblog_testing_20221207_output/')

# # print(f'[Info] Saved Data to {pipeline_output_dir}')
# # print(datetime.now())

KeyboardInterrupt: 

In [18]:
spark.stop()
sc.stop()